<a href="https://colab.research.google.com/github/jimsjoo/gilbut_portfolio/blob/master/%EB%B6%80%EB%A1%9D_%EC%9E%AC%EB%AC%B4%EC%A0%95%EB%B3%B4_%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####패키지 설치
매직명령어(!)를 사용하여 코랩리서치에 필요한 패키지를 설치한다

In [41]:
!pip install yahoo_fin
!pip install requests_html

####패키지 임포트
가장 먼저 할 일은 yahoo_fin에서 stock_info모듈을 임포트하는 것이다. 이 모듈과 함께 같이 필요한 것이  pandas패키지이다.

In [0]:
import yahoo_fin.stock_info as si
import pandas as pd

개별종목의 호가정보는 get_quote_table를 사용하여 구한다

In [0]:
quote = si.get_quote_table("aapl")

get_quote_table 메서드는 다음과 같이 여러 정보가 담긴 딕셔너리를 돌려준다.

In [44]:
quote

{'1y Target Est': 308.91,
 '52 Week Range': '170.27 - 327.85',
 'Ask': '319.00 x 1100',
 'Avg. Volume': 51660098.0,
 'Beta (5Y Monthly)': 1.17,
 'Bid': '318.91 x 900',
 "Day's Range": '315.37 - 319.23',
 'EPS (TTM)': 12.73,
 'Earnings Date': 'Jul 28, 2020 - Aug 03, 2020',
 'Ex-Dividend Date': 'May 08, 2020',
 'Forward Dividend & Yield': '3.28 (1.04%)',
 'Market Cap': '1.382T',
 'Open': 315.77,
 'PE Ratio (TTM)': 25.05,
 'Previous Close': 316.85,
 'Quote Price': 318.8900146484375,
 'Volume': 20450754.0}

다음과 같이 키 값(여기선 "PE Ratio (TTM)")을 사용하여 값을 얻을 수 있다.


In [45]:
quote["PE Ratio (TTM)"] # 22.71

25.05

PER을 구하는 또 다른 방법은 get_stats_valuation 메서드를 이용하는 것이다. 

이것은 야후파이낸스의 종목 통계섹션의 내용이다. 

In [0]:
val = si.get_stats_valuation("aapl")
val = val.iloc[:,:2]
val.columns = ["Attribute", "Recent"]

 P/E 비율을 추출하면 다음과 같다

In [47]:
float(val[val.Attribute.str.contains("Trailing P/E")].iloc[0,1])

24.77

주가매출액(Price-to-Sales) 비율

In [48]:
float(val[val.Attribute.str.contains("Price/Sales")].iloc[0,1])

5.32

####한번에 여러 재무 데이터 구하기
다우지수내 각 종목의 Price-to-Earnings 와  Price-to-Sales 비율을 구해보도록 하자. 

tickers_dow메서드는 지수를 구성하는 종목의 티커를 돌려준다.


In [0]:
# get list of Dow tickers
dow_list = si.tickers_dow() 
 
# Get data in the current column for each stock's valuation table
dow_stats = {}
for ticker in dow_list:
    temp = si.get_stats_valuation(ticker)
    temp = temp.iloc[:,:2]
    temp.columns = ["Attribute", "Recent"]
 
    dow_stats[ticker] = temp 
 
# combine all the stats valuation tables into a single data frame
combined_stats = pd.concat(dow_stats)
combined_stats = combined_stats.reset_index()
 
del combined_stats["level_1"]
 
# update column names
combined_stats.columns = ["Ticker", "Attribute", "Recent"]


주가수익률(P/E) 비율

In [50]:
# get P/E ratio for each stock
combined_stats[combined_stats.Attribute.str.contains("Trailing P/E")]

,Ticker,Attribute,Recent
2,AAPL,Trailing P/E,24.77
11,AXP,Trailing P/E,13.61
20,BA,Trailing P/E,NaN
29,CAT,Trailing P/E,12.22
38,CSCO,Trailing P/E,17.96
47,CVX,Trailing P/E,44.25
56,DIS,Trailing P/E,51.45
65,DOW,Trailing P/E,NaN
74,GS,Trailing P/E,9.77
83,HD,Trailing P/E,23.68


주가매출액비율(Price-to-Sales) 비율


In [51]:
# get P/S ratio for each stock
combined_stats[combined_stats.Attribute.str.contains("Price/Sales")]

,Ticker,Attribute,Recent
5,AAPL,Price/Sales (ttm),5.32
14,AXP,Price/Sales (ttm),2.38
23,BA,Price/Sales (ttm),1.11
32,CAT,Price/Sales (ttm),1.27
41,CSCO,Price/Sales (ttm),3.83
50,CVX,Price/Sales (ttm),1.28
59,DIS,Price/Sales (ttm),2.74
68,DOW,Price/Sales (ttm),0.64
77,GS,Price/Sales (ttm),2.02
86,HD,Price/Sales (ttm),2.32


주가순자산(Price / Book) 비율

In [52]:
# get Price-to-Book ratio for each stock
combined_stats[combined_stats.Attribute.str.contains("Price/Book")]

,Ticker,Attribute,Recent
6,AAPL,Price/Book (mrq),17.51
15,AXP,Price/Book (mrq),3.44
24,BA,Price/Book (mrq),NaN
33,CAT,Price/Book (mrq),4.41
42,CSCO,Price/Book (mrq),5.37
51,CVX,Price/Book (mrq),1.19
60,DIS,Price/Book (mrq),2.35
69,DOW,Price/Book (mrq),2.06
78,GS,Price/Book (mrq),0.76
87,HD,Price/Book (mrq),NaN


주가이익성장배율(Price / Earnings-to-Growth) 비율

In [53]:
# get PEG ratio for each stock
combined_stats[combined_stats.Attribute.str.contains("PEG")]

,Ticker,Attribute,Recent
4,AAPL,PEG Ratio (5 yr expected) 1,2.22
13,AXP,PEG Ratio (5 yr expected) 1,1.07
22,BA,PEG Ratio (5 yr expected) 1,5.91
31,CAT,PEG Ratio (5 yr expected) 1,6.52
40,CSCO,PEG Ratio (5 yr expected) 1,2.85
49,CVX,PEG Ratio (5 yr expected) 1,7.07
58,DIS,PEG Ratio (5 yr expected) 1,10.17
67,DOW,PEG Ratio (5 yr expected) 1,NaN
76,GS,PEG Ratio (5 yr expected) 1,2.70
85,HD,PEG Ratio (5 yr expected) 1,2.16


Forward P/E 비율

In [54]:
# get forward P/E ratio for each stock
combined_stats[combined_stats.Attribute.str.contains("Forward P/E")]

,Ticker,Attribute,Recent
3,AAPL,Forward P/E 1,26.74
12,AXP,Forward P/E 1,14.66
21,BA,Forward P/E 1,40.16
30,CAT,Forward P/E 1,22.57
39,CSCO,Forward P/E 1,14.93
48,CVX,Forward P/E 1,14.49
57,DIS,Forward P/E 1,60.61
66,DOW,Forward P/E 1,25.25
75,GS,Forward P/E 1,12.41
84,HD,Forward P/E 1,24.57


####여러 종목의 기타 통계 구하기
야후파이낸스 종목 통계섹션에는“Valuation Measures” 테이블이 있는데 

get_stats method를 통해 기타 통계 정보(Return on Equity (ROE), 

Return on Assets, profit margin)를 구할 수 있다.


In [0]:
dow_extra_stats = {}
for ticker in dow_list:
    dow_extra_stats[ticker] = si.get_stats(ticker)   
 
combined_extra_stats = pd.concat(dow_extra_stats)
 
combined_extra_stats = combined_extra_stats.reset_index()
 
del combined_extra_stats["level_1"]
 
combined_extra_stats.columns = ["ticker", "Attribute", "Value"]

자기자본이익률(ROE)

In [56]:
combined_extra_stats[combined_extra_stats.Attribute.str.contains("Return on Equity")]

,ticker,Attribute,Value
33,AAPL,Return on Equity (ttm),62.09%
83,AXP,Return on Equity (ttm),25.80%
133,BA,Return on Equity (ttm),NaN
183,CAT,Return on Equity (ttm),35.69%
233,CSCO,Return on Equity (ttm),29.74%
283,CVX,Return on Equity (ttm),2.51%
333,DIS,Return on Equity (ttm),5.01%
383,DOW,Return on Equity (ttm),-6.86%
433,GS,Return on Equity (ttm),8.02%
483,HD,Return on Equity (ttm),NaN


총자산이익률(ROA)

In [57]:
combined_extra_stats[combined_extra_stats.Attribute.str.contains("Return on Assets")]

,ticker,Attribute,Value
32,AAPL,Return on Assets (ttm),12.38%
82,AXP,Return on Assets (ttm),2.91%
132,BA,Return on Assets (ttm),-2.27%
182,CAT,Return on Assets (ttm),6.09%
232,CSCO,Return on Assets (ttm),9.59%
282,CVX,Return on Assets (ttm),2.49%
332,DIS,Return on Assets (ttm),2.77%
382,DOW,Return on Assets (ttm),NaN
432,GS,Return on Assets (ttm),0.74%
482,HD,Return on Assets (ttm),20.79%


이익률(profit margin)

In [58]:
combined_extra_stats[combined_extra_stats.Attribute.str.contains("Profit Margin")]

,ticker,Attribute,Value
30,AAPL,Profit Margin,21.35%
80,AXP,Profit Margin,14.63%
130,BA,Profit Margin,-4.84%
180,CAT,Profit Margin,10.41%
230,CSCO,Profit Margin,21.32%
280,CVX,Profit Margin,2.86%
330,DIS,Profit Margin,6.88%
380,DOW,Profit Margin,-4.01%
430,GS,Profit Margin,21.40%
480,HD,Profit Margin,10.20%


####재무상태표(balance sheets)
get_balance_sheet 메서드를 사용하여 재무상태표를 구할 수 있는 데, 유동현금, 자산, 부채등을 알 수 있다.


In [0]:
sheet = si.get_balance_sheet("aapl")

총현금(Total cash on hand)

In [60]:
sheet[sheet.Breakdown == "Total Cash"]

,Breakdown,9/30/2019,9/30/2018,9/30/2017,9/30/2016


자본(Stockholders’ equity)

In [61]:
sheet[sheet.Breakdown == "Total stockholders' equity"]

,Breakdown,9/30/2019,9/30/2018,9/30/2017,9/30/2016


총자산(Total Assets)

In [62]:
sheet[sheet.Breakdown == "Total Assets"]

,Breakdown,9/30/2019,9/30/2018,9/30/2017,9/30/2016
0,Total Assets,338516000,365725000,375319000,321686000


####여러 종목의 재무상태표 구하기

다우지수 구성종목들의 재무상태표를 다음과 같이 얻을 수 있다.


In [0]:
balance_sheets = {}
for ticker in dow_list:
    balance_sheets[ticker] = si.get_balance_sheet(ticker)


각 종목의 재무상태표를 묶어서  최근 데이터를 보여준다.

In [0]:
recent_sheets = {ticker : sheet.iloc[:,:2] for ticker,sheet in balance_sheets.items()}
 
for ticker in recent_sheets.keys():
    recent_sheets[ticker].columns = ["Breakdown", "Recent"]
 
# combine all balance sheets together
combined_sheets = pd.concat(recent_sheets)
 
# reset index to pull in ticker
combined_sheets = combined_sheets.reset_index()
 
# get rid of numeric index field
del combined_sheets["level_1"]
 
# update column names
combined_sheets.columns = ["Ticker", "Breakdown", "Recent"]


총자산(Total Assets)

In [65]:
combined_sheets[combined_sheets.Breakdown == "Total Assets"]

,Ticker,Breakdown,Recent
0,AAPL,Total Assets,338516000
13,AXP,Total Assets,198321000
28,BA,Total Assets,133625000
43,CAT,Total Assets,78453000
58,CSCO,Total Assets,97793000
71,CVX,Total Assets,237428000
86,DIS,Total Assets,193984000
100,DOW,Total Assets,60524000
115,GS,Total Assets,992968000
131,HD,Total Assets,51236000


####손익계산서(income statements)

손익계산서는 get_income_statement 메서드를 사용하여 얻을 수 있다


In [0]:
income=si.get_income_statement("aapl")

매출액(total revenue)

In [67]:
income[income.Breakdown == "Total Revenue"]

,Breakdown,ttm,9/30/2019,9/30/2018,9/30/2017,9/30/2016
0,Total Revenue,267981000,260174000,265595000,229234000,215639000


매출총이익(gross profit)

In [68]:
income[income.Breakdown == "Gross Profit"]

,Breakdown,ttm,9/30/2019,9/30/2018,9/30/2017,9/30/2016
2,Gross Profit,102127000,98392000,101839000,88186000,84263000


여러 종목의 손익계산서 구하기

In [0]:
income_statements = {}
for ticker in dow_list:
    income_statements[ticker] = si.get_income_statement(ticker)


각 종목의 손익계산서도 하나로 묶어  필요한 계정만 볼 수 있다

In [0]:
recent_income_statements = {ticker : sheet.iloc[:,:2] for ticker,sheet in income_statements.items()}
 
for ticker in recent_income_statements.keys():
    recent_income_statements[ticker].columns = ["Breakdown", "Recent"]
 
combined_income = pd.concat(recent_income_statements)
 
combined_income = combined_income.reset_index()
 
del combined_income["level_1"]
 
combined_income.columns = ["Ticker", "Breakdown", "Recent"]


매출액(Total Revenue)

In [0]:
combined_income[combined_income.Breakdown == "Total Revenue"]

####개별종목의 현금흐름표 다운로드

현금흐름표(cash flow statements)는 get_cash_flow 메서드를 사용하여 구한다

In [0]:
flow = si.get_cash_flow("aapl")

여러 종목의 현금흐름표 다운로드

In [0]:
cash_flows = {}
for ticker in dow_list:
    cash_flows[ticker] = si.get_cash_flow(ticker)


다운받은 현금흐름표 묶기

In [0]:
recent_cash_flows = {ticker : flow.iloc[:,:2] for ticker,flow in cash_flows.items()}
  
for ticker in recent_cash_flows.keys():
    recent_cash_flows[ticker].columns = ["Breakdown", "Recent"]
  
combined_cash_flows = pd.concat(recent_cash_flows)
 
combined_cash_flows = combined_cash_flows.reset_index()
 
del combined_cash_flows["level_1"]
 
combined_cash_flows.columns = ["Ticker", "Breakdown", "Recent"]


잉여현금흐름(free cash flow)

In [0]:
combined_cash_flows[combined_cash_flows.Breakdown == "Free Cash Flow"]

부채정보(debt information)



In [0]:
combined_cash_flows[combined_cash_flows.Breakdown == "Issuance of Debt"]